<a id='top'></a>
## Introduction: <br>
#### Hi welcome to this notebook in this notebook we are gonna do analysis of papers with state of art text analytics methods by using COVID-19 Open Research Dataset.<br>

* What is coronavirus disease 2019? <br>
Coronavirus disease 2019 (COVID-19) is a respiratory illness that can spread from person to person. The virus that causes COVID-19 is a novel coronavirus that was first identified during an investigation into an outbreak in Wuhan, China. <br>

* Can I get COVID-19? <br>
Unfortunately Yes. COVID-19 is spreading from person to person in parts of the world. Risk of infection from the virus that causes COVID-19 is higher for people who are close contacts of someone known to have COVID-19, for example healthcare workers, or household members. Other people at higher risk for infection are those who live in or have recently been in an area with ongoing spread of COVID-19. <br>

* How does COVID-19 spread? <br>
The virus that causes COVID-19 probably emerged from an animal source, but is now spreading from person to person. The virus is thought to spread mainly between people who are in close contact with one another (within about 6 feet) through respiratory droplets produced when an infected person coughs or sneezes. It also may be possible that a person can get COVID-19 by touching a surface or object that has the virus on it and then touching their own mouth, nose, or possibly their eyes, but this is not thought to be the main way the virus spreads.

* Is there a vaccine? <br>
There is currently no vaccine to protect against COVID-19. The best way to prevent infection is to take everyday preventive actions, like avoiding close contact with people who are sick and washing your hands often. <br>

* Is there a treatment? <br>
There is no specific antiviral treatment for COVID-19.People with COVID-19 can seek medical care to help relieve symptoms. <br>

#### Dont get panic and be safe.



<h1 style="text-align:center;font-size:200%;;">Don't Panic</h1>
<img src="https://media2.giphy.com/media/Y4WQSe2EucDEEUCKWu/giphy.gif" width="300" height="300" align="center"/>

<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Notebook Content:</h3>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#one_" role="tab" aria-controls="profile">1. Importing DataFrames<span class="badge badge-primary badge-pill">1</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#two_" role="tab" aria-controls="messages">2. Word Clouds of Authors<span class="badge badge-primary badge-pill">2</span></a>
  <a class="list-group-item list-group-item-action"  data-toggle="list" href="#three_" role="tab" aria-controls="settings">3. Word Clouds of Affiliations<span class="badge badge-primary badge-pill">3</span></a>
   <a class="list-group-item list-group-item-action"  data-toggle="list" href="#four_" role="tab" aria-controls="settings">4. Ngram Analysis<span class="badge badge-primary badge-pill">4</span></a>
   <a class="list-group-item list-group-item-action"  data-toggle="list" href="#five_" role="tab" aria-controls="settings">5. Topic modelling<span class="badge badge-primary badge-pill">5</span></a>
    <a class="list-group-item list-group-item-action"  data-toggle="list" href="#six_" role="tab" aria-controls="settings">6. Readability tests on Abstracts<span class="badge badge-primary badge-pill">6</span></a>
    <a class="list-group-item list-group-item-action"  data-toggle="list" href="#seven_" role="tab" aria-controls="settings">7. Summarising Texts to Understand the content Fastly<span class="badge badge-primary badge-pill">7</span></a>
    
    
  

<font color='red'>If you find this kernel useful please consider upvoting it 😊 which keeps me motivated for doing hard work and to produce more quality content.</font>

In [ ]:
!pip install textstat
!pip install chart_studio

In [ ]:
import string
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
from tqdm import tqdm
%matplotlib inline


import nltk
nltk.download('punkt') # one time execution
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize   
stop_words = stopwords.words('english')
remove_words = set(stopwords.words('english')) 

from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import textstat

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from gensim.summarization.summarizer import summarize
from spacy.lang.en import English
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from statistics import *
import concurrent.futures

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

punctuations = string.punctuation
stopwords = list(STOP_WORDS)

parser = English()

def plot_readability(a,title,bins=0.1,colors=['#3A4750']):
    #reference and credits : https://www.kaggle.com/thebrownviking20/analyzing-quora-for-the-insinceres
    trace1 = ff.create_distplot([a], [" Abstract "], bin_size=bins, colors=colors, show_rug=False)
    trace1['layout'].update(title=title)
    iplot(trace1, filename='Distplot')
    table_data= [["Statistical Measures","Abstract"],
                ["Mean",mean(a)],
                ["Standard Deviation",pstdev(a)],
                ["Variance",pvariance(a)],
                ["Median",median(a)],
                ["Maximum value",max(a)],
                ["Minimum value",min(a)]]
    trace2 = ff.create_table(table_data)
    iplot(trace2, filename='Table')
    
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

parser = English()
def spacy_tokenizer(sentence):
    #reference and credits : https://www.kaggle.com/thebrownviking20/analyzing-quora-for-the-insinceres
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens


#references and credits : https://www.geeksforgeeks.org/print-colors-python-terminal/
def prRed(skk): print("\033[91m {}\033[00m" .format(skk)) 
def prGreen(skk): print("\033[92m {}\033[00m" .format(skk)) 
def prYellow(skk): print("\033[93m {}\033[00m" .format(skk)) 
def prLightPurple(skk): print("\033[94m {}\033[00m" .format(skk)) 
def prPurple(skk): print("\033[95m {}\033[00m" .format(skk)) 
def prCyan(skk): print("\033[96m {}\033[00m" .format(skk)) 
def prLightGray(skk): print("\033[97m {}\033[00m" .format(skk)) 
def prBlack(skk): print("\033[98m {}\033[00m" .format(skk)) 

import os
import warnings
warnings.filterwarnings('ignore')

<a id='one_'></a>
# 1. Importing DataFrames

In [ ]:
#data set credits : https://www.kaggle.com/xhlulu/cord-19-eda-parse-json-and-generate-clean-csv
biorxiv_data = pd.read_csv('/kaggle/input/cord-19-eda-parse-json-and-generate-clean-csv/biorxiv_clean.csv')
clean_comm_data = pd.read_csv('/kaggle/input/cord-19-eda-parse-json-and-generate-clean-csv/clean_comm_use.csv')
clean_noncomm_data = pd.read_csv('/kaggle/input/cord-19-eda-parse-json-and-generate-clean-csv/clean_noncomm_use.csv')
clean_pmc_data = pd.read_csv('/kaggle/input/cord-19-eda-parse-json-and-generate-clean-csv/clean_pmc.csv')

In [ ]:
biorxiv_data.head(3)

In [ ]:
clean_comm_data.head(3)

In [ ]:
clean_noncomm_data.head(3)

In [ ]:
clean_pmc_data.head(3)

<a id='two_'></a>
# 2. Word Clouds of Authors

## 2.1 Word Cloud of Authors in biorxiv medrxiv Data

In [ ]:
from wordcloud import WordCloud, STOPWORDS

# Thanks : https://www.kaggle.com/aashita/word-clouds-of-various-shapes ##
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=50, figure_size=(15.0,15.0), 
                   title = None, title_size=20, image_color=False,color = color):
    stopwords = set(STOPWORDS)
    more_stopwords = {'one', 'br', 'Po', 'th', 'sayi', 'fo', 'Unknown'}
    stopwords = stopwords.union(more_stopwords)

    wordcloud = WordCloud(background_color=color,
                    stopwords = stopwords,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    width=800, 
                    height=400,
                    mask = mask)
    wordcloud.generate(str(text))
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout()  
    
plot_wordcloud(biorxiv_data['authors'].values, title="Word Cloud of Authors in biorxiv medrxiv Data",color = 'black')

## 2.2 Word Cloud of Authors in comm use subset Data

In [ ]:
plot_wordcloud(clean_comm_data['authors'].values, title="Word Cloud of Authors in comm use subset Data",color = 'white')

## 2.3 Word Cloud of Authors in Non common use subset Data

In [ ]:
plot_wordcloud(clean_noncomm_data['authors'].values, title="Word Cloud of Authors in Non common use subset Data",color = 'red')

## 2.4 Word Cloud of Authors in pmc_custom Data

In [ ]:
plot_wordcloud(clean_pmc_data['authors'].values, title="Word Cloud of Authors pmc_custom Data",color = 'violet')

<a id='three_'></a>
# 3. Word Clouds of Affiliations

## 3.1 Word Cloud of Affiliations in biorxiv medrxiv Data

In [ ]:
plot_wordcloud(biorxiv_data['affiliations'].values, title="Word Cloud of Affiliations in biorxiv medrxiv Data ",color = 'green')

## 3.2 Word Cloud of Affiliations in comm use subset Data

In [ ]:
plot_wordcloud(clean_comm_data['affiliations'].values, title="Word Cloud of Affiliations in comm use subset Data",color = 'orange')

## 3.3 Word Cloud of Affiliations in Non common use subset Data

In [ ]:
plot_wordcloud(clean_noncomm_data['affiliations'].values, title="Word Cloud of affiliations in Non common use subset Data",color = 'brown')

## 3.4 Word Cloud of Affiliations in pmc_custom Data

In [ ]:
plot_wordcloud(clean_pmc_data['affiliations'].values, title="Word Cloud of Affiliations in pmc_custom Data",color = 'gray')

<a id='four_'></a>
# 4. Ngram Analysis

## 4.1 Ngram Analysis of Abstracts

### 4.1.1 Word Frequency analysis of abstracts

In [ ]:
df1 = biorxiv_data['abstract'].dropna()
df3 = clean_comm_data["abstract"].dropna()
df2 = clean_noncomm_data["abstract"].dropna()
df4 = clean_pmc_data["abstract"].dropna()

## custom function for ngram generation ##
def generate_ngrams(text, n_gram=1):
    #Reference and credits: https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-qiqc
    token = [token for token in text.lower().split(" ") if token != "" if token not in STOPWORDS]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [" ".join(ngram) for ngram in ngrams]

## custom function for horizontal bar chart ##
def horizontal_bar_chart(df, color):
    trace = go.Bar(
        y=df["word"].values[::-1],
        x=df["wordcount"].values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        ),
    )
    return trace

## Get the bar chart from sincere questions ##
freq_dict = defaultdict(int)
for sent in df1:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(25), 'green')

## Get the bar chart from insincere questions ##
freq_dict = defaultdict(int)
for sent in df2:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace1 = horizontal_bar_chart(fd_sorted.head(25), 'orange')

freq_dict = defaultdict(int)
for sent in df3:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace2 = horizontal_bar_chart(fd_sorted.head(25), 'black')

freq_dict = defaultdict(int)
for sent in df4:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace3 = horizontal_bar_chart(fd_sorted.head(25), 'red')



# Creating two subplots
fig = tools.make_subplots(rows=2, cols=2, vertical_spacing=0.04,
                          subplot_titles=["Frequent words in biorxiv_data", 
                                          "Frequent words in clean_comm_data",
                                          "Frequent words in clean_noncomm_data",
                                          "Frequent words in clean_pmc_data"])
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 2, 2)



fig['layout'].update(height=1000, width=800, paper_bgcolor='rgb(233,233,233)', title="Word Count Plots of Abstracts")
iplot(fig, filename='word-plots')


## Observations: 
* We can see from above word frequency plots most the top words consits of Virus,Respiratory,disease,Transmission etc..,

### 4.1.2 Bigram analysis of Abstracts

In [ ]:
freq_dict = defaultdict(int)
for sent in df1:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(25), 'gray')


freq_dict = defaultdict(int)
for sent in df2:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace1 = horizontal_bar_chart(fd_sorted.head(25), 'orange')

freq_dict = defaultdict(int)
for sent in df3:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace2 = horizontal_bar_chart(fd_sorted.head(25), 'brown')


freq_dict = defaultdict(int)
for sent in df4:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace3 = horizontal_bar_chart(fd_sorted.head(25), 'pink')


# Creating two subplots
fig = tools.make_subplots(rows=2, cols=2, vertical_spacing=0.04,horizontal_spacing=0.25,
                          subplot_titles=["Frequent words in biorxiv_data", 
                                          "Frequent words in clean_comm_data",
                                          "Frequent words in clean_noncomm_data",
                                          "Frequent words in clean_pmc_data"])
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 2, 2)


fig['layout'].update(height=1000, width=800, paper_bgcolor='rgb(233,233,233)', title="Bigram Count Plots of Abstracts")
iplot(fig, filename='word-plots')

## Observations:
* We can observe from above bigram plot that most of words contains Respiratory syndrome as we know corona will effect our respiratory system.

### 4.1.3 Trigram analysis of Abstracts

In [ ]:
for sent in df1:
    for word in generate_ngrams(sent,3):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(25), 'blue')


freq_dict = defaultdict(int)
for sent in df2:
    for word in generate_ngrams(sent,3):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace1 = horizontal_bar_chart(fd_sorted.head(25), 'green')

freq_dict = defaultdict(int)
for sent in df3:
    for word in generate_ngrams(sent,3):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace2 = horizontal_bar_chart(fd_sorted.head(25), 'violet')

freq_dict = defaultdict(int)
for sent in df4:
    for word in generate_ngrams(sent,3):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace3 = horizontal_bar_chart(fd_sorted.head(25), 'red')




# Creating two subplots
fig = tools.make_subplots(rows=4, cols=1, vertical_spacing=0.04, horizontal_spacing=0.05,
                          subplot_titles=["Frequent words in biorxiv_data", 
                                          "Frequent words in clean_comm_data",
                                          "Frequent words in clean_noncomm_data",
                                          "Frequent words in clean_pmc_data"])
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 2, 1)
fig.append_trace(trace2, 3, 1)
fig.append_trace(trace3, 4, 1)
fig['layout'].update(height=1200, width=1200, paper_bgcolor='rgb(233,233,233)', title="Trigram Count Plots")
iplot(fig, filename='word-plots')

* The plots says it all!

## 4.2 Ngram Analysis of Text in papers

### 4.2.1 Word frequency Plots of texts in papers

In [ ]:

df1 = biorxiv_data['text'].dropna()
df3 = clean_comm_data["text"].dropna()
df2 = clean_noncomm_data["text"].dropna()
df4 = clean_pmc_data["text"].dropna()

## custom function for ngram generation ##
def generate_ngrams(text, n_gram=1):
    token = [token for token in text.lower().split(" ") if token != "" if token not in STOPWORDS]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [" ".join(ngram) for ngram in ngrams]

## custom function for horizontal bar chart ##
def horizontal_bar_chart(df, color):
    trace = go.Bar(
        y=df["word"].values[::-1],
        x=df["wordcount"].values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        ),
    )
    return trace

## Get the bar chart from sincere questions ##
freq_dict = defaultdict(int)
for sent in df1:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(25), 'orange')

## Get the bar chart from insincere questions ##
freq_dict = defaultdict(int)
for sent in df2:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace1 = horizontal_bar_chart(fd_sorted.head(25), 'green')

freq_dict = defaultdict(int)
for sent in df3:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace2 = horizontal_bar_chart(fd_sorted.head(25), 'blue')

freq_dict = defaultdict(int)
for sent in df4:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace3 = horizontal_bar_chart(fd_sorted.head(25), 'red')



# Creating two subplots
fig = tools.make_subplots(rows=2, cols=2, vertical_spacing=0.04,
                          subplot_titles=["Frequent words in biorxiv_data", 
                                          "Frequent words in clean_comm_data",
                                          "Frequent words in clean_noncomm_data",
                                          "Frequent words in clean_pmc_data"])
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 2, 2)



fig['layout'].update(height=1000, width=800, paper_bgcolor='rgb(233,233,233)', title="Word Count Plots of Texts in papers")
iplot(fig, filename='word-plots')


### 4.2.2 Bigram Plots of texts in papers

In [ ]:
freq_dict = defaultdict(int)
for sent in df1:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(25), 'yellow')


freq_dict = defaultdict(int)
for sent in df2:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace1 = horizontal_bar_chart(fd_sorted.head(25), 'red')

freq_dict = defaultdict(int)
for sent in df3:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace2 = horizontal_bar_chart(fd_sorted.head(25), 'brown')


freq_dict = defaultdict(int)
for sent in df4:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace3 = horizontal_bar_chart(fd_sorted.head(25), 'blue')


# Creating two subplots
fig = tools.make_subplots(rows=2, cols=2, vertical_spacing=0.04,horizontal_spacing=0.25,
                          subplot_titles=["Frequent words in biorxiv_data", 
                                          "Frequent words in clean_comm_data",
                                          "Frequent words in clean_noncomm_data",
                                          "Frequent words in clean_pmc_data"])
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 2, 2)


fig['layout'].update(height=1000, width=800, paper_bgcolor='rgb(233,233,233)', title="Bigram Count Plots of Texts in papers")
iplot(fig, filename='word-plots')

### Obseravtions:
* We can observe papers text contains matter about t-cells.T cells are also called T lymphocyte, type of leukocyte (white blood cell) that is an essential part of the immune system. T cells are one of two primary types of lymphocytes—B cells being the second type—that determine the specificity of immune response to antigens (foreign substances) in the body.
* Also about Influenza Virus, commonly known as "the flu", is an infectious disease caused by an influenza virus. Symptoms can be mild to severe. The most common symptoms include: high fever, runny nose, sore throat, muscle and joint pain, headache, coughing, and feeling tired.Which are also the symptoms of Corona Virus.

<a id='five_'></a>
# 5. Topic modelling.
* Topic modeling is a machine learning technique that automatically analyzes text data to determine cluster words for a set of documents. This is known as ‘unsupervised’ machine learning because it doesn’t require a predefined list of tags or training data that’s been previously classified by humans. <br>
* Topic modeling involves counting words and grouping similar word patterns to infer topics within unstructured data. <br>
* Read More : [Here](https://monkeylearn.com/blog/introduction-to-topic-modeling/) <br>

In [ ]:
%%time
text_1 = biorxiv_data["abstract"].dropna().apply(spacy_tokenizer)
text_2 = clean_comm_data["abstract"].dropna().apply(spacy_tokenizer)
text_3 = clean_noncomm_data['abstract'].dropna().apply(spacy_tokenizer)
text_4 = clean_pmc_data['abstract'].dropna().apply(spacy_tokenizer)
#count vectorization
vectorizer_1= CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
vectorizer_2= CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
vectorizer_3 =  CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
vectorizer_4= CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')

text1_vectorized = vectorizer_1.fit_transform(text_1)
text2_vectorized = vectorizer_2.fit_transform(text_2)
text3_vectorized = vectorizer_3.fit_transform(text_3)
text4_vectorized = vectorizer_4.fit_transform(text_4)

In [ ]:
%%time
lda1 = LatentDirichletAllocation(n_components=5, max_iter=5, learning_method='online',verbose=True)
lda2= LatentDirichletAllocation(n_components=5, max_iter=5, learning_method='online',verbose=True)
lda3 = LatentDirichletAllocation(n_components=5, max_iter=5, learning_method='online',verbose=True)
lda4 = LatentDirichletAllocation(n_components=5, max_iter=5, learning_method='online',verbose=True)

lda_1 = lda1.fit_transform(text1_vectorized)
lda_2 = lda2.fit_transform(text2_vectorized)
lda_3 = lda3.fit_transform(text3_vectorized)
lda_4 = lda4.fit_transform(text4_vectorized)

In [ ]:
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [ ]:
print("LDA Model of Bioarvix data Abstracts:")
selected_topics(lda1, vectorizer_1)

In [ ]:
print("LDA Model of clean_comm data Abstracts:")
selected_topics(lda2, vectorizer_2)

In [ ]:
print("LDA Model of clean_noncomm data Abstracts:")
selected_topics(lda3, vectorizer_3)

In [ ]:
print("LDA Model of clean_pmc data Abstracts:")
selected_topics(lda4, vectorizer_4)

In [ ]:
%%time
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda1, text1_vectorized, vectorizer_1, mds='tsne')
dash

### Key Observations: <br>
#### We can infer from the above Topic modelling is that: <br>
1. The topic 1 is about Epidemiology.Epidemiology is the branch of medicine which deals with the incidence, distribution, and possible control of diseases and other factors relating to health. <br>
2. The topic 2 is more about Protein,Human Cells,Genes also drug tests on Mice. <br>
3. Topic 3 is towards genetic and bioinformatic. <br>
4. Topic 4 is about Pneumonia,Respiratory diseases and clinical medicine. <br>
5. Topic 5 is about SARS(Severe Acute Respiratory Syndrome),nCov (novel corono virus),Mers (Middle East respiratory syndrome-related coronavirus),Epitope(he part of an antigen that is recognized by the immune system, specifically by antibodies, B cells, or T cells). 

In [ ]:
%%time
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda2, text2_vectorized, vectorizer_2, mds='tsne')
dash

### Key Observations: <br>
1. Topic 1 is towards Virues,Protein,how human immune system damages,also how humans react to drugs. <br>
2. Topic 2 is about public health,epidemic(a widespread occurrence of an infectious disease in a community at a particular time),Trasmission of Covid. <br>
3. Topic 3 is closely related to Human Gene Sequencing,Pedv(Porcine epidemic diarrhea virus). <br>
4. Topic 4 is about how to detect Covid-19 and how to Diagnosis the Covid-19. <br>
5. I think topic 5 is about how to detect the Covid-19 we can see that there was more about how Covid will effect the Respiratory system. <br>


In [ ]:
%%time
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda3, text3_vectorized, vectorizer_3, mds='tsne')
dash

### Key Observations: <br>
1. Topic 1 is related to protein expression,Gene expression and viruses. <br>
2. Topic 2 is about how Covid-19 is effecting humans with respect to Age,also which human parts are more effected(respiratory system). <br>
3. Topic 3 is about Related research , Public health. <br> 
4. Topic 4 is closely related to Coronvirus effects on humans also how it is transmitted. <br>
5. The significant increase in Covid cases and treatment was related to Topic 5. <br>

In [ ]:
%%time
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda4, text4_vectorized, vectorizer_4, mds='tsne')
dash

### Key Observations: 
1. Topic 1 infers to the Outbreak of Covid,Risk,Cure and its dangerous effects on human system. <br>
2. Topic 2 is related to Gene Sequences. <br>
3. Topic 3 is towards Symptoms of Patients who tested positive also about HBov (Human bocavirus is the name given to all viruses in the genus Bocaparvovirus of virus family Parvoviridae that are known to infect humans). <br>
4. Topic 4 is about zoonotic disease(A zoonosis is an infectious disease caused by bacteria, viruses, or parasites that spread from non-human animals to humans). <br>
5. Topic 5 Related to animal experiment. <br>

#### Therefore we can say that Topic modelling will definitely help paper readers and researchers to understand current research.

In [ ]:
del text_1,text_2,text_3,text_4,vectorizer_1,vectorizer_2,vectorizer_3,vectorizer_4,text1_vectorized,text2_vectorized ,text3_vectorized ,text4_vectorized 

 <a id='six_'></a>
 # 6. Readability tests on Abstracts

### Readability is the ease with which a reader can understand a written text. In natural language processing, the readability of text depends on its content. It focuses on the words we choose, and how we put them into sentences and paragraphs for the readers to comprehend.

## 6.1 The Flesch Reading Ease formula <br>
* In the Flesch reading-ease test, higher scores indicate material that is easier to read; lower numbers mark passages that are more difficult to read. The formula for the Flesch reading-ease score (FRES) test is
![](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd4916e193d2f96fa3b74ee258aaa6fe242e110e)

## Score - Difficulty <br>
* 90-100 - Very Easy
* 80-89 - Easy
* 70-79 - Fairly Easy
* 60-69 - Standard
* 50-59 - Fairly Difficult
* 30-49 - Difficult
* 0-29 - Very Confusing

Read More : [Wikipedia](https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests)

In [ ]:
tqdm.pandas()
fre_1 = np.array(biorxiv_data["abstract"].dropna().apply(textstat.flesch_reading_ease))
plot_readability(fre_1,"Flesch Reading Ease",20)

In [ ]:
tqdm.pandas()
fre_2 = np.array(clean_comm_data["abstract"].dropna().apply(textstat.flesch_reading_ease))
plot_readability(fre_2,"Flesch Reading Ease",20,colors = ['#8D99AE'] )

In [ ]:
tqdm.pandas()
fre_3 = np.array(clean_noncomm_data["abstract"].dropna().apply(textstat.flesch_reading_ease))
plot_readability(fre_3,"Flesch Reading Ease",20,colors = ['#C65D17'])

In [ ]:
tqdm.pandas()
fre_4 = np.array(clean_pmc_data["abstract"].dropna().apply(textstat.flesch_reading_ease))
plot_readability(fre_4,"Flesch Reading Ease",20,colors = ['#DDB967'])

### Observations: <br>
*  From the above Flesch Reading Ease formula on abstracts, we can obsereve that the mean value was around 26 for all the data which infers that it is bit confusing to read and understand the abstracts.

## 6.2 Dale-Chall Readability Score <br>
* Different from other tests, since it uses a lookup table of the most commonly used 3000 English words. Thus it returns the grade level using the New Dale-Chall Formula. <br>
* The formula for calculating the raw score of the Dale–Chall readability score is given below: <br>
![](https://wikimedia.org/api/rest_v1/media/math/render/svg/0541f1e629f0c06796c5a5babb3fac8d100a858c)

* Score - Understood by

** 4.9 or lower - average 4th-grade student or lower ** <br>
** 5.0–5.9 - average 5th or 6th-grade student ** <br>
** 6.0–6.9 - average 7th or 8th-grade student ** <br>
** 7.0–7.9 - average 9th or 10th-grade student ** <br>
** 8.0–8.9 - average 11th or 12th-grade student ** <br>
** 9.0–9.9 - average 13th to 15th-grade (college) student and above** <br>

* Read More : [Wikipedia](https://en.wikipedia.org/wiki/Dale%E2%80%93Chall_readability_formula)

In [ ]:
dcr_ = np.array(biorxiv_data["abstract"].dropna().apply(textstat.dale_chall_readability_score))
plot_readability(dcr_,"Dale-Chall Readability Score",1,['#C65D17'])

In [ ]:
dcr_ = np.array(clean_comm_data["abstract"].dropna().apply(textstat.dale_chall_readability_score))
plot_readability(dcr_,"Dale-Chall Readability Score",1,['#DDB967'])

In [ ]:
dcr_ = np.array(clean_noncomm_data["abstract"].dropna().apply(textstat.dale_chall_readability_score))
plot_readability(dcr_,"Dale-Chall Readability Score",1,['#8D99AE'])

In [ ]:
dcr_ = np.array(clean_pmc_data["abstract"].dropna().apply(textstat.dale_chall_readability_score))
plot_readability(dcr_,"Dale-Chall Readability Score",1,['#EF233C'])

### Observations:
* From above Dale-Chall Readability Score tests we can observe that mean score was around ~9.7 which means these abstracts can understood by high grade students.

## 6.3 Automated Readability Index <br>
* Returns the ARI (Automated Readability Index) which outputs a number that approximates the grade level needed to comprehend the text.For example if the ARI is 6.5, then the grade level to comprehend the text is 6th to 7th grade.
* Formula to calculate ARI: <br>
![](https://wikimedia.org/api/rest_v1/media/math/render/svg/878d1640d23781351133cad73bdf27bdf8bfe2fd) <br>
* Read More: [Wikipedia](https://en.wikipedia.org/wiki/Automated_readability_index)

In [ ]:
ari_ = np.array(biorxiv_data["abstract"].dropna().apply(textstat.coleman_liau_index))
plot_readability(ari_,"Automated Readability Index",10,['#2B2D42'])

In [ ]:
ari_ = np.array(clean_comm_data["abstract"].dropna().apply(textstat.coleman_liau_index))
plot_readability(ari_,"Automated Readability Index",10,['#FF934F'])

In [ ]:
ari_ = np.array(clean_noncomm_data["abstract"].dropna().apply(textstat.coleman_liau_index))
plot_readability(ari_,"Automated Readability Index",10,['#488286'])

In [ ]:
ari_ = np.array(clean_pmc_data["abstract"].dropna().apply(textstat.coleman_liau_index))
plot_readability(ari_,"Automated Readability Index",10,['#8491A3'])

### Observations: <br>
* We can observe from above Automated Readability Index Plots that the mean values are about close to ~16.0 which means that the person who holds college degree or higher can understand the papers and their contents.

## 6.4 The Coleman-Liau Index <br>
* Returns the grade level of the text using the Coleman-Liau Formula. This is a grade formula in that a score of 9.3 means that a ninth grader would be able to read the document. <br>
* The Coleman–Liau index is calculated with the following formula: <br>
![](https://wikimedia.org/api/rest_v1/media/math/render/svg/cae44bbb96eaaca26e6aaf3b65c342f69f3d49ce) <br>
L is the average number of letters per 100 words and S is the average number of sentences per 100 words. <br>
* Read More : [Wikipedia](https://en.wikipedia.org/wiki/Coleman%E2%80%93Liau_index)

In [ ]:
cli_ = np.array(biorxiv_data["abstract"].dropna().apply(textstat.coleman_liau_index))
plot_readability(cli_,"The Coleman-Liau Index",10,['#e8434e'])

In [ ]:
cli_ = np.array(clean_comm_data["abstract"].dropna().apply(textstat.coleman_liau_index))
plot_readability(cli_,"The Coleman-Liau Index",10,['#a36f72'])

In [ ]:
cli_ = np.array(clean_noncomm_data["abstract"].dropna().apply(textstat.coleman_liau_index))
plot_readability(cli_,"The Coleman-Liau Index",10,['#2d5c5a'])

In [ ]:
cli_ = np.array(clean_pmc_data["abstract"].dropna().apply(textstat.coleman_liau_index))
plot_readability(cli_,"The Coleman-Liau Index",10,['#64c48c'])

### Observations: <br>
* The above Coleman-Liau Index histogram plots says that the mean values of CLI was about ~16 which means the person with the age of about 25 years who completed his degree can read and understand the papers.

## 6.5 The Fog Scale (Gunning FOG Formula) <br>
* In linguistics, the Gunning fog index is a readability test for English writing. The index estimates the years of formal education a person needs to understand the text on the first reading. For instance, a fog index of 12 requires the reading level of a United States high school senior (around 18 years old). <br>
* The formula to calculate Fog scale: <br>
![](https://wikimedia.org/api/rest_v1/media/math/render/svg/84cd504cf61d43230ef59fbd0ecf201796e5e577)
* Read more : [Wikipedia](https://en.wikipedia.org/wiki/Gunning_fog_index)

In [ ]:
fog_ = np.array(biorxiv_data["abstract"].dropna().apply(textstat.gunning_fog))
plot_readability(fog_,"The Fog Scale (Gunning FOG Formula)",4,['#d98714'])

In [ ]:
fog_ = np.array(clean_comm_data["abstract"].dropna().apply(textstat.gunning_fog))
plot_readability(fog_,"The Fog Scale (Gunning FOG Formula)",4,['#7dd609'])

In [ ]:
fog_ = np.array(clean_noncomm_data["abstract"].dropna().apply(textstat.gunning_fog))
plot_readability(fog_,"The Fog Scale (Gunning FOG Formula)",4,['#E2D58B'])

In [ ]:
fog_ = np.array(clean_pmc_data["abstract"].dropna().apply(textstat.gunning_fog))
plot_readability(fog_,"The Fog Scale (Gunning FOG Formula)",4,['#612620'])

### Obseravtions: <br>
* Here plots Speak more than words!! The graphs says it all.

<a id='seven_'></a>
# 7. Summarising Texts to Understand the content Fastly

In [ ]:
#lets make one dataframe.
all_data = pd.concat([biorxiv_data,clean_comm_data,clean_noncomm_data,clean_pmc_data]).reset_index()

In [ ]:
text = all_data['text'][10]
print('Text before summarizations:')
prCyan(text)

Yes, we can see there is alot of text here.So,what we can do is summarising the text so that its get easier to paper readers,researchers to fastly read and under stand the information in that text.

In [ ]:
print('Text after summarization:')
prPurple(summarize(text,word_count = 250))

We can see from the above output that the text was summarised to 250 words (you can choose on your own). Now its very easy to read and grasp the content in the text. It will be defenitely useful for the researchers and paper readers to grasp the content very fastly.

## Thanks for reading.Alot more to come stay tuned and be safe 😊. <br>
<font color='red'>If you find this kernel useful please consider upvoting it 😊 which keeps me motivated for doing hard work and to produce more quality content.</font>


<h1 style="text-align:center;font-size:125%;;">Follow these steps and protect yourself and your loving ones</h1>
<img src="https://global.unitednations.entermediadb.net/assets/mediadb/services/module/asset/downloads/preset/Libraries/Graphics+Library/29-01-2020-WHO-CoronaVirus-anim-public-advice.gif/image1170x530cropped.jpg" width="500" height="500" align="center"/>

<a href="#top" class="btn btn-primary btn-lg active" role="button" aria-pressed="true">Go to TOP</a>